In [1]:
#%pip install -U torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113

In [2]:
#%pip install -U transformers

In [3]:
#%pip install -U datasets

In [4]:
#%pip install -U accelerate

In [5]:
#%pip install -U sentencepiece

In [6]:
import torch
torch.cuda.is_available()

True

In [7]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2023-04-27 04:25:43.306630: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-27 04:25:46.362005: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-27 04:25:46.362963: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-04-27 04:25:46.362983: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

Num GPUs Available:  4


2023-04-27 04:25:48.336118: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-27 04:25:48.339278: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-27 04:25:48.342292: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-27 04:25:48.345337: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-27 04:25:48.431973: I tensorflow/compiler/xla/stream_executo

In [8]:
#%pip install bitsandbytes-cuda113
#%pip install bitsandbytes

In [9]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset

In [10]:
# directory where to cache models
cache_dir = '/home/jupyter/data/transformers'

## Pre-Trained Model Examples

In [11]:
def generate(prompt, max_new_tokens=100):
    
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    
    result = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    return "\n".join(result)

In [12]:
# one of the available flan-t5 models
checkpoint = "google/flan-ul2"

In [14]:
# loading model and tokenizer based on particular model checkpoint
# tokenizer is used to preprocess text input in a way that the model can understand
tokenizer = AutoTokenizer.from_pretrained(checkpoint, device_map="auto")
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, device_map="auto", load_in_8bit=True, cache_dir=cache_dir)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.
/opt/conda/lib/python3.7/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /opt/conda did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/opt/conda/lib/python3.7/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/share/ca-certificates,/etc/ssl/certs,/etc/ca-certificates/update.d')}
  warn(msg)
/opt/conda/lib/python3.7/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/proxy/%PORT%')}
  warn(msg)
/opt/conda/lib/python3.7/site-packages/bit


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.7/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /opt/conda/lib/python3.7/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [15]:
prompt = "translate English to German: How old are you?"

response = generate(prompt)
response

'Wie alt sind Sie?'

In [16]:
prompt = "summarize: The pizza at Roberts Pizza & Dough company is great, although expensive. I'd recommended the pepporoni."

response = generate(prompt, max_new_tokens=10)
response

'The pepporoni is great.'

In [17]:
prompt = "summarize: The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 10295.35 is added to the Public Contract Code, to read:\n10295.35.\n(a) (1) Notwithstanding any other law, a state agency shall not enter into any contract for the acquisition of goods or services in the amount of one hundred thousand dollars ($100,000) or more with a contractor that, in the provision of benefits, discriminates between employees on the basis of an employee’s or dependent’s actual or perceived gender identity, including, but not limited to, the employee’s or dependent’s identification as transgender.\n(2) For purposes of this section, “contract” includes contracts with a cumulative amount of one hundred thousand dollars ($100,000) or more per contractor in each fiscal year.\n(3) For purposes of this section, an employee health plan is discriminatory if the plan is not consistent with Section 1365.5 of the Health and Safety Code and Section 10140 of the Insurance Code.\n(4) The requirements of this section shall apply only to those portions of a contractor’s operations that occur under any of the following conditions:\n(A) Within the state.\n(B) On real property outside the state if the property is owned by the state or if the state has a right to occupy the property, and if the contractor’s presence at that location is connected to a contract with the state.\n(C) Elsewhere in the United States where work related to a state contract is being performed.\n(b) Contractors shall treat as confidential, to the maximum extent allowed by law or by the requirement of the contractor’s insurance provider, any request by an employee or applicant for employment benefits or any documentation of eligibility for benefits submitted by an employee or applicant for employment.\n(c) After taking all reasonable measures to find a contractor that complies with this section, as determined by the state agency, the requirements of this section may be waived under any of the following circumstances:\n(1) There is only one prospective contractor willing to enter into a specific contract with the state agency."

response = generate(prompt, max_new_tokens=100)
response

'A bill to amend the Public Contract Code to prohibit discrimination in the provision of benefits by contractors with respect to employees and dependents on the basis of an employee’s or dependent’s actual or perceived gender identity, including, but not limited to, the employee’s or dependent’s identification as transgender, and for other purposes.'

In [18]:
prompt = "A step by step recipe to make bolognese pasta do not repeat the steps:"

response = generate(prompt, max_new_tokens=100)
response

'In a large skillet, brown the ground beef and sausage over medium heat. Add the onion, carrots, and celery and cook until tender. Add the garlic and cook for 1 minute. Add the wine and cook until reduced by half. Add the tomatoes, tomato paste, basil, oregano, salt, and pepper. Simmer for 30 minutes. Add the pasta and toss to coat. Serve.'

In [19]:
prompt = (
    "Question: What class do I TA for in the past? "
    "Context: My name is Ignas and I currently a TA NLP and Conversation AI but have also TA'd Optimization and Simulation Methods in the past"
)

response = generate(prompt, max_new_tokens=100)
response

'Optimization'